In [12]:
import pandas as pd
import math
import os
import csv
import random
import numpy as np
from sklearn import linear_model
from sklearn.model_selection import cross_val_score
from collections import Counter

In [13]:
data_dir = 'data'
team_dict = {'Utah': 'Utah Jazz', 'Memphis': 'Memphis Grizzlies', 'San Antonio': 'San Antonio Spurs',
        'Indiana': 'Indiana Pacers', 'Atlanta': 'Atlanta Hawks', 'Miami': 'Miami Heat',
        'Charlotte': 'Charlotte Hornets', 'Milwaukee': 'Milwaukee Bucks', 'Chicago': 'Chicago Bulls',
        'Washington': 'Washington Wizards', 'Portland Trail': 'Portland Trail Blazers',
        'New Orleans': 'New Orleans Pelicans', 'Cleveland': 'Cleveland Cavaliers', 'Detroit': 'Detroit Pistons',
        'Golden State': 'Golden State Warriors', 'Los Angeles C': 'Los Angeles Clippers', 
        'Houston': 'Houston Rockets', 'Brooklyn': 'Brooklyn Nets', 'Toronto': 'Toronto Raptors',
        'Philadelphia': 'Philadelphia 76ers', 'New York': 'New York Knicks', 'Boston': 'Boston Celtics',
        'Orlando': 'Orlando Magic', 'Oklahoma City': 'Oklahoma City Thunder', 'Dallas': 'Dallas Mavericks',
        'Phoenix': 'Phoenix Suns', 'Denver': 'Denver Nuggets', 'Sacramento': 'Sacramento Kings',
        'Los Angeles L': 'Los Angeles Lakers', 'Minnesota': 'Minnesota Timberwolves'
        }

In [14]:
class raw_data_handler:
    raw_result = 'Result_raw.csv'
    raw_dict = {'Mstat' : 'Miscellaneous_Stat_raw.csv',
                'Ostat' : 'Opponent_Per_Game_Stat_raw.csv',
                'Tstat' : 'Team_Per_Game_Stat_raw.csv'}
    csv_dict = {'Mstat': 'Miscellaneous_Stat.csv',
                'Ostat': 'Opponent_Per_Game_Stat.csv',
                'Tstat': 'Team_Per_Game_Stat_raw.csv'}
    
    # function for normalizing the team name
    def get_team(self, row):
        # handle the Los Angeles Lakers and Los Angeles Clippers
        if 'Los' in row:
            team = row[:13]
        else:
            team = ' '.join(row.split(" ")[:-1])
        return team_dict.get(team, np.nan)
  
    # helper function for creating the csv file
    def create_csv(self, df, csv_name, season):
        csv_path = os.path.join(data_dir, season, csv_name)
        df.to_csv(csv_path, encoding = 'utf8', index = False)
    
    # helper function for creating the csv file for result
    def create_result_csv(self, df, season):
        result_path = os.path.join(data_dir, season, 'Result.csv')
        data = []
        for index, row in df.iterrows():
            data.append([row['Visitor/Neutral'], row['Home/Neutral'], 'V'] 
                        if row['PTS'] > row['PTS.1'] else [row['Home/Neutral'], row['Visitor/Neutral'], 'H'])
        with open(result_path, 'w') as f:
            writer = csv.writer(f)
            writer.writerow(['WTeam', 'LTeam', 'WLoc'])
            writer.writerows(data)
    
    # helper function for normalize stat data
    def norm_store_stat(self, stat_name, season):
        stat = pd.read_csv(os.path.join(data_dir, season, self.raw_dict[stat_name]))
        stat['Team'] = stat['Team'].apply(self.get_team)
        self.create_csv(stat.dropna(), self.csv_dict[stat_name], season)
    
    def run(self, season_num):
        season = "{}-{}".format(season_num, season_num + 1)
        
        # read and normalize the team name in Rstat
        Rstat = pd.read_csv(os.path.join(data_dir, season, self.raw_result))
        Rstat['Visitor/Neutral'] = Rstat['Visitor/Neutral'].apply(self.get_team)
        Rstat['Home/Neutral'] = Rstat['Home/Neutral'].apply(self.get_team)
        # create the result file
        self.create_result_csv(Rstat, season)
        
        # create the schedule file
        self.create_csv(Rstat.loc[:,['Visitor/Neutral', 'Home/Neutral']], 
                        'Schedule.csv', season)
        
        # read, normalize and store the Mstat
        self.norm_store_stat('Mstat', season)
        
        # read, normalize and store the Ostat
        self.norm_store_stat('Ostat', season)
        
        # read, normalize and store the Tstat
        self.norm_store_stat('Tstat', season)

         

In [21]:
class team_win_lose_predictor:
    base_elo = 1600
    
    # get the fit regression model
    def get_model(self, season_num):
        season = "{}-{}".format(season_num, season_num + 1)
        Mstat = pd.read_csv(os.path.join(data_dir, season, 'Miscellaneous_Stat.csv')).drop(['Rk', 'Arena'], axis=1)
        Ostat = pd.read_csv(os.path.join(data_dir, season, 'Opponent_Per_Game_Stat.csv')).drop(['Rk', 'G', 'MP'], axis=1)
        Tstat = pd.read_csv(os.path.join(data_dir, season, 'Team_Per_Game_Stat.csv')).drop(['Rk', 'G', 'MP'], axis=1)
        Rstat = pd.read_csv(os.path.join(data_dir, season, 'Result.csv'))
        
        team_stats = Mstat.merge(Ostat, how='left', on='Team').merge(Tstat, how='left', on='Team')
        self.team_stats = team_stats.set_index('Team', inplace=False, drop=True)

        self.team_elos = dict([(i, self.base_elo) for i in team_dict.values()])
        
        X, y = self.build_dataset(Rstat)
        print("Fitting on %d game samples.." % len(X))
        model = linear_model.LogisticRegression()
        model.fit(X, y)
        
        print("Doing cross-validation..")
        print(cross_val_score(model, X, y, cv = 10, scoring='accuracy', n_jobs=-1).mean())
        
        return model
    
    # update the elo value for the given teams and their win-lose relationship
    def calc_elo(self, win_team, lose_team):
        winner_rank = self.team_elos[win_team]
        loser_rank = self.team_elos[lose_team]

        rank_diff = winner_rank - loser_rank
        odds = 1 / (1 + 10 ** (-rank_diff / 400))
        if winner_rank < 2100:
            k = 32
        elif winner_rank >= 2100 and winner_rank < 2400:
            k = 24
        else:
            k = 16
        new_winner_rank = round(winner_rank + (k * (1 - odds)))
        new_loser_rank = loser_rank + winner_rank - new_winner_rank
        return new_winner_rank, new_loser_rank
    
    # build the training dataset based on the raw data
    def build_dataset(self, raw_data):
        print("Building dataset ...")
        X = []
        y = []
        for index, row in raw_data.iterrows():
            Wteam = row['WTeam']
            Lteam = row['LTeam']

            team1_elo = self.team_elos[Wteam]
            team2_elo = self.team_elos[Lteam]

            if row['WLoc'] == 'H':
                team1_elo += 100
            else:
                team2_elo += 100

            team1_features = [team1_elo]
            team2_features = [team2_elo]

            for key, value in self.team_stats.loc[Wteam].iteritems():
                team1_features.append(value)
            for key, value in self.team_stats.loc[Lteam].iteritems():
                team2_features.append(value)

            if random.random() > 0.5:
                X.append(team1_features + team2_features)
                y.append(0)
            else:
                X.append(team2_features + team1_features)
                y.append(1)

            new_winner_rank, new_loser_rank = self.calc_elo(Wteam, Lteam)
            self.team_elos[Wteam] = new_winner_rank
            self.team_elos[Lteam] = new_loser_rank
        
        return np.nan_to_num(X), y
    
    # predict the winner of a match
    def predict_winner(self, team_1, team_2, model):
        features = []
        
        features.append(self.team_elos[team_1])
        for key, value in self.team_stats.loc[team_1].iteritems():
            features.append(value)

        features.append(self.team_elos[team_2] + 100)
        for key, value in self.team_stats.loc[team_2].iteritems():
            features.append(value)

        features = np.nan_to_num(features)
        return model.predict_proba([features])
    
    # predict the win-lose relationship with the given schedule
    def predict_schedule(self, schedule, model):
        result = []
        for index, row in schedule.iterrows():
            team1 = row['Visitor/Neutral']
            team2 = row['Home/Neutral']
            pred = self.predict_winner(team1, team2, model)
            prob = pred[0][0]
            item = [team1, team2, prob] if prob > 0.5 else [team2, team1, 1 - prob]
            result.append(item)
        return result
    
    # check the accuracy of the predict result against the true match result
    def check_accuracy(self, result, next_season):
        with open(os.path.join(data_dir, next_season, 'Result.csv')) as f:
            true_result = list(map(lambda x: x.split(',')[0:-1], f.readlines()))[1:len(result)+1]
            conf = [0.5, 0.6, 0.7, 0.8, 0.9]
            error = dict([(i, 0) for i in conf])
            team_list = list(team_dict.values())
            # create the columns of the accuracy dataframe
            columns = np.append(np.array(['{}/mis_W {}/mis_L'.format(i, i).split(' ') for i in conf]).flatten(), ['pred_W', 'pred_L', 'true_W', 'true_L'])
            # initialize the accuracy dataframe
            team_stat = pd.DataFrame(data = 0, index = team_list, columns = columns)
            # for each match, update the accuracy dataframe with the true and pred result
            for t, p in zip(true_result, result):
                team_stat.loc[p[0], 'pred_W'] += 1
                team_stat.loc[p[1], 'pred_L'] += 1
                team_stat.loc[t[0], 'true_W'] += 1
                team_stat.loc[t[1], 'true_L'] += 1
                # pred winner doesn't match the true result
                if t[0] != p[0]:
                    # update each error number
                    for i in error:
                        if p[2] >= i:
                            error[i] += 1
                            team_stat.loc[p[0], '{}/mis_W'.format(i)] += 1
                            team_stat.loc[p[1], '{}/mis_L'.format(i)] += 1
            team_stat.to_csv(os.path.join(data_dir, next_season, 'Team_Stat_Accu.csv'))
            for i in error:
                print('{} confidence error rate: {}'.format(i*100, error[i]/len([r[2]>=i for r in result])))
    
    # run the predict
    def run(self, season_num):
        model = self.get_model(season_num)
        next_season = "{}-{}".format(season_num + 1, season_num + 2)
        schedule = pd.read_csv(os.path.join(data_dir, next_season, 'Schedule.csv'))
        result = self.predict_schedule(schedule, model)
#         print(Counter([r[0] for r in result]))
        with open(os.path.join(data_dir, next_season, 'Pre_Result.csv'), 'w') as f:
            writer = csv.writer(f)
            writer.writerow(['win', 'lose', 'probability'])
            writer.writerows(result)
            print('done.')
        
        self.check_accuracy(result, next_season)
    
    

        
        

In [22]:
data_handler = raw_data_handler()
predictor = team_win_lose_predictor()

In [ ]:
data_handler.run(13)

In [25]:
predictor.run(16)

Building dataset ...
Fitting on 1309 game samples..
Doing cross-validation..
0.6553943130660688
done.
50.0 confidence error rate: 0.37110016420361247
60.0 confidence error rate: 0.2438423645320197
70.0 confidence error rate: 0.12397372742200329
80.0 confidence error rate: 0.04433497536945813
90.0 confidence error rate: 0.008210180623973728
